# Import Library

In [34]:
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Load Dataset

In [3]:
df = pd.read_csv("/kaggle/input/combined-faq-answers/Combined_Chatbot_Dataset.csv")
df.head()

,text,intent,answer
0,Bagaimana cara mengedit biodata (profile)?,T1,Untuk mengedit biodata atau profil Anda di Job...
1,Bagaimana cara membuat CV ATS generated?,T3,JobRise menyediakan fitur pembuatan CV yang di...
2,Bagaimana cara apply jobs?,T2,"Untuk melamar pekerjaan di JobRise, ikuti lang..."
3,Bagaimana cara save job?,T4,Anda dapat menyimpan lowongan pekerjaan yang m...
4,Bagaimana cara cek status lamaran?,T2,Untuk memeriksa status lamaran pekerjaan yang ...


# EDA

In [4]:
print("Jumlah data:", len(df))
print("\nKolom yang tersedia:", df.columns.tolist())
df.info()

Jumlah data: 17

Kolom yang tersedia: ['text', 'intent', 'answer']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17 entries, 0 to 16
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    17 non-null     object
 1   intent  17 non-null     object
 2   answer  17 non-null     object
dtypes: object(3)
memory usage: 540.0+ bytes


In [5]:
df['intent'].value_counts()

intent
T1    3
T2    3
T4    3
N6    2
T3    1
T6    1
N1    1
N2    1
N3    1
N5    1
Name: count, dtype: int64

# Load Model using Word2Vec

In [18]:
sentences = df['text'].dropna().apply(word_tokenize).tolist()

# Train Data

In [19]:
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Testing Local

In [20]:
def sentence_vector(sentence, model):
    words = word_tokenize(sentence)
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

In [32]:
question_vectors = [sentence_vector(q, w2v_model) for q in df['text']]

def predict_intent(user_input):
    user_vec = sentence_vector(user_input, w2v_model).reshape(1, -1)
    similarities = cosine_similarity(user_vec, question_vectors)[0]
    best_idx = similarities.argmax()
    return {
        "user_input": user_input,
        "matched_question": df['text'].iloc[best_idx],
        "matched_intent": df['intent'].iloc[best_idx],
        "response": df['answer'].iloc[best_idx],
        "similarity_score": round(float(similarities[best_idx]), 3)
    }

print(predict_intent("Apa itu Jobrise"))


{'user_input': 'Apa itu Jobrise', 'matched_question': 'Apa keunggulan JobRise dibandingkan platform pencarian kerja lainnya?', 'matched_intent': 'N6', 'response': 'JobRise memiliki beberapa keunggulan kompetitif yang membedakannya dari platform pencarian kerja lainnya:\n\n1. Teknologi AI dan Machine Learning:\n- Sistem rekomendasi pekerjaan yang lebih akurat berdasarkan analisis komprehensif profil\n- Prediksi tren karir dan kecocokan posisi berdasarkan big data\n\n2. Tools Canggih untuk Persiapan Karir:\n- CV builder dengan optimasi ATS yang lebih canggih\n\n3. Memperhatikan aspek psikologis\n- JobRise tidak hanya berfokus pada informasi lowongan pekerjaan, tetapi juga memperhatikan kondisi mental maupun emosional dari user.\n\nDengan kombinasi keunggulan ini, JobRise tidak hanya membantu Anda menemukan pekerjaan, tetapi juga mendukung pengembangan karir jangka panjang Anda secara komprehensif.', 'similarity_score': 0.385}


# Export model

In [33]:
w2v_model.save("w2v_model.model")

# Save Data Embeddings

In [35]:
question_vectors = [sentence_vector(q, w2v_model) for q in df['text']]

data_to_save = {
    "questions": df['text'].tolist(),
    "answers": df['answer'].tolist(),
    "intents": df['intent'].tolist(),
    "question_embeddings": np.array(question_vectors)
}

with open("embeddings.pkl", "wb") as f:
    pickle.dump(data_to_save, f)
